In [3]:
options(warn=-1)

suppressMessages(library(ArchR))
suppressMessages(library(Seurat))
suppressMessages(library(Signac))
suppressMessages(library(harmony))
suppressMessages(library(dplyr))
suppressMessages(library(cowplot))
suppressMessages(library(harmony))
suppressMessages(library(Nebulosa))
suppressMessages(library(ggpubr))
suppressMessages(library(Ipaper))
suppressMessages(library(phateR))
suppressMessages(library(parallel))
suppressMessages(library(ggrepel))
suppressMessages(library(circlize))

In [4]:
set.seed(42)
getwd()

addArchRThreads(threads = parallel::detectCores() - 2)
addArchRGenome("hg38")

[1] "/data/scATA/SingleCellOpenChromatin/local/VisiumHeartRevision/IntegrativeAnalysis/Cardiomyocyte/code"

Setting default number of Parallel threads to 126.

Setting default genome to Hg38.



In [3]:
## subset atac project
proj <- loadArchRProject("../data/snATAC_healthy_intermediate_damaged", showLogo = FALSE)

Successfully loaded ArchRProject!



In [4]:
## we first plot the activity and expression of the selected TFs
df_tf <- read.csv("../data/snATAC_healthy_intermediate_damaged/sel_tf_by_expression.csv", row.names = 1)

,idx1,idx2,matchname1,matchname2,name1,name2,Correlation,VarAssay1,VarAssay2,TStat,Pval,FDR
,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,9,6062,RORA,RORA,z:RORA_9,chr15:RORA,0.3588709,0.584518167,0.9259434,3.806181,2.458795e-04,0.005185823
4,16,10193,REL,REL,z:REL_16,chr2:REL,0.1525305,0.426540284,0.6444339,1.527853,1.297703e-01,0.431544539
5,17,3251,RELA,RELA,z:RELA_17,chr11:RELA,0.1030552,0.420221169,0.5125618,1.025655,3.075792e-01,0.659329184
8,24,11533,NFATC2,NFATC2,z:NFATC2_24,chr20:NFATC2,0.1059962,0.731437599,0.7432461,1.055253,2.939041e-01,0.646310540
12,32,10042,FOSL2,FOSL2,z:FOSL2_32,chr2:FOSL2,0.3869445,0.995260664,0.9255230,4.154151,6.982884e-05,0.002025036
13,33,17643,FOXH1,FOXH1,z:FOXH1_33,chr8:FOXH1,0.2004729,0.007898894,0.9359298,2.025703,4.551371e-02,0.242739770


In [7]:
## obtain tf activity along the trajectory
trajMM  <- getTrajectory(ArchRProj = proj,
                         name = "healthy_intermediate_damaged",
                         useMatrix = "MotifMatrix",
                         log2Norm = FALSE,
                         scaleTo = NULL,
                        smoothWindow = 11)

trajMM <- trajMM[!grepl("deviations", rownames(trajMM)), ]

matMM <- plotTrajectoryHeatmap(trajMM, 
                            varCutOff = 0,
                            pal = paletteContinuous(set = "solarExtra"),
                            limits = c(-2, 2),
                              returnMatrix = TRUE)

df_motif_activity <- as.data.frame(matMM[df_tf$name1, ]) 
colnames(df_motif_activity) <- 1:100
df_motif_activity$tf <- rownames(df_motif_activity)

df_motif_activity2 <- tidyr::pivot_longer(df_motif_activity, -tf, names_to = "pseudotime", values_to = "tf_activity")

df_motif_activity2$pseudotime <- as.numeric(df_motif_activity2$pseudotime)
df_motif_activity2$tf <- stringr::str_split_fixed(df_motif_activity2$tf, ":|_", 3)[, 2]

Creating Trajectory Group Matrix..

Smoothing...

ArchR logging to : ArchRLogs/ArchR-plotTrajectoryHeatmap-3f6ff53dbd05b4-Date-2022-02-09_Time-18-40-26.log
If there is an issue, please report to github with logFile!

useSeqnames is NULL or greater than 1 with a Sparse.Assays.Matrix trajectory input.

2022-02-09 18:40:26 : 

force=FALSE thus continuing with subsetting useSeqnames = z

2022-02-09 18:40:26 : 

Preparing Main Heatmap..

'magick' package is suggested to install to give better rasterization.

Set `ht_opt$message = FALSE` to turn off this message.

ArchR logging successful to : ArchRLogs/ArchR-plotTrajectoryHeatmap-3f6ff53dbd05b4-Date-2022-02-09_Time-18-40-26.log



tf,pseudotime,tf_activity
<chr>,<dbl>,<dbl>
RORA,1,-0.6714971
RORA,2,-0.6714971
RORA,3,-0.6714971
RORA,4,-0.6714971
RORA,5,-0.6714971
RORA,6,-0.6714971


In [8]:
trajGEX <- getTrajectory(ArchRProj = proj,
                         name = "healthy_intermediate_damaged",
                         useMatrix = "GeneExpressionMatrix",
                         log2Norm = TRUE,
                        smoothWindow = 11)

trajGEX <- trajGEX[df_tf$name2, ]

gene_expression <- plotTrajectoryHeatmap(trajGEX,
                        varCutOff = 0,
                        pal = paletteContinuous(set = "horizonExtra"),
                        limits = c(-2, 2),
                                         returnMatrix = TRUE)

df_gene_expression <- as.data.frame(gene_expression) 
colnames(df_gene_expression) <- 1:100
df_gene_expression$tf <- rownames(df_gene_expression)

df_gene_expression2 <- tidyr::pivot_longer(df_gene_expression, -tf, names_to = "pseudotime", values_to = "tf_expression")

df_gene_expression2$pseudotime <- as.numeric(df_gene_expression2$pseudotime)
df_gene_expression2$tf <- stringr::str_split_fixed(df_gene_expression2$tf, ":", 2)[, 2]

Creating Trajectory Group Matrix..

Smoothing...

ArchR logging to : ArchRLogs/ArchR-plotTrajectoryHeatmap-3f6ff5620ce895-Date-2022-02-09_Time-18-42-12.log
If there is an issue, please report to github with logFile!

Preparing Main Heatmap..

'magick' package is suggested to install to give better rasterization.

Set `ht_opt$message = FALSE` to turn off this message.

ArchR logging successful to : ArchRLogs/ArchR-plotTrajectoryHeatmap-3f6ff5620ce895-Date-2022-02-09_Time-18-42-12.log



In [49]:
if(!dir.exists("../data/snATAC_healthy_intermediate_damaged/VisGRN_V2/TF")){
    dir.create("../data/snATAC_healthy_intermediate_damaged/VisGRN_V2/TF", recursive = TRUE)
}

write.csv(df_motif_activity2, "../data/snATAC_healthy_intermediate_damaged/VisGRN_V2/TF_activity.csv")
write.csv(df_gene_expression2, "../data/snATAC_healthy_intermediate_damaged/VisGRN_V2/TF_expression.csv")

for(sel_tf in unique(df_motif_activity2$tf)){
        df_plot1 <- subset(df_motif_activity2, tf == sel_tf) %>%
            rename(value = tf_activity) %>%
            mutate(value = scale(value)) %>%
            mutate(data = "Activity")

        
        df_plot2 <- subset(df_gene_expression2, tf == sel_tf) %>%
            rename(value = tf_expression)%>%
            mutate(value = scale(value)) %>%
            mutate(data = "Expression")
        
        df_plot <- rbind(df_plot1, df_plot2)
        
        p <- ggplot(data = df_plot, aes(x = pseudotime, y = value, color = data)) +
        geom_smooth(method = "loess",se = FALSE) +
        theme_cowplot() +
        theme(legend.title = element_blank())
    
         ggsave(glue::glue("../data/snATAC_healthy_intermediate_damaged/VisGRN_V2/TF/{sel_tf}.pdf"),
                  plot = p, height = 3, width = 5)
        
}

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using for

In [1]:
df_tf_gene <- read.csv("../data/snATAC_healthy_intermediate_damaged/gene_regulatory_network.csv")

#df_tf_gene$tf <- stringr::str_split_fixed(df_tf_gene$tf, ":", 2)[, 2]

head(df_tf_gene)
nrow(df_tf_gene)

,tf,gene,correlation
,<chr>,<chr>,<dbl>
1,NFIA,TRDN,0.5352448
2,FOXH1,TRDN,0.7274919
3,HLF,TRDN,0.7511693
4,ESRRB,TRDN,0.5484806
5,RFX3,KCNMB2,0.8387288
6,NR3C2,SPHKAP,0.6645800


[1] 12009

In [2]:
trajGEX <- getTrajectory(ArchRProj = proj,
                         name = "healthy_intermediate_damaged",
                         useMatrix = "GeneExpressionMatrix",
                         log2Norm = TRUE,
                        smoothWindow = 11)

ERROR: Error in getTrajectory(ArchRProj = proj, name = "healthy_intermediate_damaged", : could not find function "getTrajectory"


In [ ]:
gene_expression <- plotTrajectoryHeatmap(trajGEX,
                                         varCutOff = 0.1,
                                         limits = c(-2, 2),
                                         returnMatrix = TRUE)

rownames(gene_expression) <- stringr::str_split_fixed(rownames(gene_expression), ":", 2)[, 2]

head(gene_expression)

In [ ]:
df_gene_expression <- as.data.frame(gene_expression) 
colnames(df_gene_expression) <- 1:100
df_gene_expression$gene <- rownames(df_gene_expression)
df_gene_expression2 <- tidyr::pivot_longer(df_gene_expression, -gene, 
                                           names_to = "pseudotime", values_to = "gene_expression")
df_gene_expression2$pseudotime <- as.numeric(df_gene_expression2$pseudotime)

write.csv(df_gene_expression2, "../data/snATAC_healthy_intermediate_damaged/VisGRN_V2/TF_Target_Gene_expression.csv")

In [ ]:
if(!dir.exists(glue::glue("../data/snATAC_healthy_intermediate_damaged/VisGRN_V2/TF_Target_Expression"))){
        dir.create(glue::glue("../data/snATAC_healthy_intermediate_damaged/VisGRN_V2/TF_Target_Expression"), 
                   recursive = TRUE)
}

for(name1 in unique(df_tf_gene$gene)){
        df_plot <- subset(df_gene_expression2, gene == name1)

        p <- ggplot(df_plot, aes(x = pseudotime, y = gene_expression)) +
         geom_smooth(method = "loess",se = FALSE) +
        theme_cowplot() + ggtitle(name1)
    
        ggsave(glue::glue("../data/snATAC_healthy_intermediate_damaged/VisGRN_V2/TF_Target_Expression/{name1}.pdf"),
              plot = p, height = 4, width = 5)

    }

In [ ]:
# for(gene_name in unique(df_tf_gene$gene)){
# #for(tf_name in c("KLF2_403")){
# df_tf_gene_sub <- subset(df_tf_gene, tf == tf_name)
    
#     trajGEX_sub <- trajGEX[unique(df_tf_gene_sub$gene), ]
    
#     gene_expression <- plotTrajectoryHeatmap(trajGEX_sub,
#                         varCutOff = 0,
#                         pal = paletteContinuous(set = "horizonExtra"),
#                         limits = c(-2, 2),
#                                          returnMatrix = TRUE)
    
#     df_gene_expression <- as.data.frame(gene_expression) 
#     colnames(df_gene_expression) <- 1:100
#     df_gene_expression$tf <- rownames(df_gene_expression)

#     df_gene_expression2 <- tidyr::pivot_longer(df_gene_expression, -tf, 
#                                                names_to = "pseudotime", values_to = "tf_expression")

#     df_gene_expression2$pseudotime <- as.numeric(df_gene_expression2$pseudotime)
#     df_gene_expression2$tf <- stringr::str_split_fixed(df_gene_expression2$tf, ":", 2)[, 2]



#     for(name1 in unique(df_gene_expression2$tf)){
#         df_plot <- subset(df_gene_expression2, tf == name1)

#         p <- ggplot(df_plot, aes(x = pseudotime, y = tf_expression)) +
#             geom_point(aes(color = pseudotime)) + 
#             scale_color_gradientn(colours = ArchR::paletteContinuous(set = "blueYellow")) +
#          geom_smooth(method = "loess",se = FALSE) +
#         theme_cowplot() + ggtitle(name1)


#         ggsave(glue::glue("../data/snATAC_healthy_intermediate_damaged/VisGRN/TF_Target_Expression/{name1}.pdf"),
#               plot = p, height = 4, width = 5)

#     }
    

# }